# UKB MODELS

In [1]:
!pip install statsmodels
!pip install lifelines==0.26.4

In [2]:
# Imports here.
import numpy as np
import pandas as pd
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import fdrcorrection
from lifelines import CoxPHFitter

import warnings
warnings.filterwarnings("ignore")

In [ ]:
!dx download -r 'data/files_for_cox'

In [3]:
#Set variables
STUDY_ENDS = '2023-09-30'
STUDY_START = '1999-01-01'

In [4]:
#Get list of codes from prep notebook
ndd_list = ['AD', 'DEM', 'PD', 'VAS', 'MS', 'ALS']
codes = ['G47_DATE', 'F51_DATE', 'G47.3_DATE']
lags = ['0', '0-1', '1-5', '5-10', '10-15', '5-15']

In [6]:
#Get list of people with Welsh data -- datafield 26426
w = pd.read_csv('files_for_cox/Wales_dep_n21182_participant.csv')
wales_list = list(w['eid'])

In [7]:
timeline = '1999-2023'
model = 'COX'

results = []

for lag in lags:

    for ndd in ndd_list:

        #Load df
        df = pd.read_csv(f'files_for_cox/{ndd}_with_tenure_lags_45.csv', parse_dates = True)
 
        #Include this line in to exclude Welsh data
        #df = df[~df['ID'].isin(wales_list)]

        #Only selected codes with data
        codes_with_data = []
        for code in codes:
            m = df[['TOWNSEND', 'AGE_OF_RECRUIT', 'GENETIC_SEX', 'tenure', ndd, f'QC{lag}_' +code]]
            n=sum(m[f'QC{lag}_'+code])
            df_pair = m[m[f'QC{lag}_'+code]==1]
            n_pairs = sum(df_pair[ndd])
            if n == 0:
                pass
            elif n_pairs < 5:
                pass
            elif n == n_pairs:
                covariate = code
                HR = np.nan
                ci_min = np.nan
                ci_max = np.nan
                p = np.nan
                print(covariate, ndd, HR, ci_min, ci_max, p, n_pairs, n)
                results.append((covariate, ndd, model, timeline, lag, HR, ci_min, ci_max, p, n_pairs, n))
                pass
            else:
                codes_with_data.append(code)

        for code in codes_with_data:
            m = df[['TOWNSEND', 'AGE_OF_RECRUIT', 'GENETIC_SEX', 'tenure', ndd, f'QC{lag}_' +code]]
            n=sum(m[f'QC{lag}_'+code])
            df_pair = m[m[f'QC{lag}_'+code]==1]
            n_pairs = sum(df_pair[ndd])

            cph = CoxPHFitter()
            cph.fit(m, duration_col = 'tenure', event_col = ndd, show_progress=False, step_size = 0.1)
            #cph.print_summary()
            #cph.plot()

            actual_p = cph._compute_p_values()
            results_df = cph.summary
            results_df = results_df.reset_index()
            test = results_df.iloc[3]

            covariate = code
            HR = test['exp(coef)']
            ci_min = test['exp(coef) lower 95%']
            ci_max = test['exp(coef) upper 95%']
            p = actual_p[3]

            print(covariate, ndd, lag, HR, ci_min, ci_max, p, n_pairs, n)
            results.append((covariate, ndd, model, timeline, lag, HR, ci_min, ci_max, p, n_pairs, n))
            
cox1 = pd.DataFrame(results, columns=('PRIOR','OUTCOME', 'MODEL','TIMELINE', 'LAG', 'HR', 'ci_min', "ci_max", 'P_VAL', "N_pairs", "N"))

G47_DATE AD 0 1.3520739929482262 1.1358733593233823 1.6094259693668016 0.0006911155914964988 132 6971
G47.3_DATE AD 0 1.2853341206865339 1.0137505393274246 1.6296748930926384 0.038197178189569275 70 3937
G47_DATE DEM 0 1.7287939680117974 1.5599654389306048 1.9158941020403786 1.6093341858826532e-25 385 7219
F51_DATE DEM 0 2.684373331539521 1.5890402419522145 4.534724793519284 0.00022319535795006535 14 159
G47.3_DATE DEM 0 1.6844657571850312 1.4700782844735385 1.930118223700633 6.025641367305257e-14 215 4078
G47_DATE PD 0 1.4322283531476747 1.1967144935665106 1.7140914283128428 8.886361675844944e-05 125 6967
F51_DATE PD 0 4.530147763518941 2.3545479057895684 8.71599966552126 6.04661591582171e-06 9 154
G47.3_DATE PD 0 1.440430527150501 1.1460873285121633 1.8103682432651986 0.001753482545994215 76 3945
G47_DATE VAS 0 2.0684569522428387 1.6905873941498173 2.53078555896447 1.6463658527220729e-12 101 6943
F51_DATE VAS 0 6.140078349231444 2.922481873697444 12.900186815189041 1.6577707459695538

In [8]:
#Combine results
output = pd.concat([cox1])
output = output[output['N_pairs'] > 4]

#Adding FDR Correction

#Sort P-values
output = output.sort_values(by = "P_VAL")

#Drop Nan-values
output = output.dropna()

#FDR Correction
rejected, p_corr = fdrcorrection(output['P_VAL'], is_sorted=True)
output['P_CORR'] = p_corr
output['REJECTED'] = rejected

output

,PRIOR,OUTCOME,MODEL,TIMELINE,LAG,HR,ci_min,ci_max,P_VAL,N_pairs,N,P_CORR,REJECTED
15,G47_DATE,DEM,COX,1999-2023,0-1,8.349764,6.016191,11.588487,6.722014e-37,36,181,4.302089e-35,True
16,G47.3_DATE,DEM,COX,1999-2023,0-1,9.747574,6.766508,14.041984,2.259577e-34,29,112,7.230647e-33,True
2,G47_DATE,DEM,COX,1999-2023,0,1.728794,1.559965,1.915894,1.609334e-25,385,7219,3.433246e-24,True
17,G47_DATE,PD,COX,1999-2023,0-1,10.348344,6.419929,16.680594,8.519510e-22,17,162,1.363122e-20,True
19,G47_DATE,VAS,COX,1999-2023,0-1,14.136191,8.005290,24.962483,6.873642e-20,12,157,8.798262e-19,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,G47_DATE,MS,COX,1999-2023,0,1.249894,0.769723,2.029605,3.671523e-01,17,6861,3.916292e-01,False
40,G47_DATE,ALS,COX,1999-2023,5-10,1.360899,0.563777,3.285068,4.931268e-01,5,1520,5.173790e-01,False
32,G47.3_DATE,AD,COX,1999-2023,5-10,1.066715,0.652583,1.743658,7.967203e-01,16,1062,8.224210e-01,False
12,G47.3_DATE,MS,COX,1999-2023,0,0.960653,0.454685,2.029654,9.162310e-01,7,3877,9.307743e-01,False


In [9]:
codes = pd.read_csv('files_for_cox/sleep_ICD10_codes_with_description.csv')
codes['DATE'] = codes['ICD10'] + '_DATE'
codes

,ICD10,Description,DATE
0,G47.0,Insomnia,G47.0_DATE
1,G47.1,Hypersomnia,G47.1_DATE
2,G47.2,Circadian rthythm sleep disorder,G47.2_DATE
3,G47.3,Sleep apnea,G47.3_DATE
4,G47.4,Narcolepsy and cataplexy,G47.4_DATE
5,G47.5,Parasomnia,G47.5_DATE
6,G47.6,Sleep related movement disorders,G47.6_DATE
7,G47.8,Other sleep disorders,G47.8_DATE
8,G47.9,"Sleep disorder, unspecified",G47.9_DATE
9,F51.0,Insomnia not due to a substance or known physi...,F51.0_DATE


In [10]:
final_results = output.merge(codes, left_on='PRIOR', right_on = 'DATE', how='left')
final_results['COHORT'] = 'UKB'
final_results = final_results.rename(columns = {'REJECTED':'SIGNIFICANT'})
final_results.loc[final_results.LAG == '0', 'TIMELINE'] = 'all'
final_results.loc[final_results.LAG == '0-1', 'TIMELINE'] = '< 1 year before tenure'
final_results.loc[final_results.LAG == '1-5', 'TIMELINE'] = '1-5 years before tenure'
final_results.loc[final_results.LAG == '5-10', 'TIMELINE'] = '5-10 years before tenure'
final_results.loc[final_results.LAG == '10-15', 'TIMELINE'] = '10-15 years before tenure'
final_results.loc[final_results.LAG == '5-15', 'TIMELINE'] = '5-15 years before tenure'
final_results = final_results[['ICD10', 'PRIOR', 'Description', 'OUTCOME', 'COHORT', 'LAG', 'TIMELINE', 'HR', 'ci_min', 'ci_max', 'P_VAL',
       'N_pairs', 'N', 'P_CORR', 'SIGNIFICANT']]
final_results = final_results.drop(columns = 'PRIOR')
final_results = final_results.rename(columns = {'Description':'PRIOR'})
final_results

,ICD10,PRIOR,OUTCOME,COHORT,LAG,TIMELINE,HR,ci_min,ci_max,P_VAL,N_pairs,N,P_CORR,SIGNIFICANT
0,G47,Other sleep disorders,DEM,UKB,0-1,< 1 year before tenure,8.349764,6.016191,11.588487,6.722014e-37,36,181,4.302089e-35,True
1,G47.3,Sleep apnea,DEM,UKB,0-1,< 1 year before tenure,9.747574,6.766508,14.041984,2.259577e-34,29,112,7.230647e-33,True
2,G47,Other sleep disorders,DEM,UKB,0,all,1.728794,1.559965,1.915894,1.609334e-25,385,7219,3.433246e-24,True
3,G47,Other sleep disorders,PD,UKB,0-1,< 1 year before tenure,10.348344,6.419929,16.680594,8.519510e-22,17,162,1.363122e-20,True
4,G47,Other sleep disorders,VAS,UKB,0-1,< 1 year before tenure,14.136191,8.005290,24.962483,6.873642e-20,12,157,8.798262e-19,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,G47,Other sleep disorders,MS,UKB,0,all,1.249894,0.769723,2.029605,3.671523e-01,17,6861,3.916292e-01,False
60,G47,Other sleep disorders,ALS,UKB,5-10,5-10 years before tenure,1.360899,0.563777,3.285068,4.931268e-01,5,1520,5.173790e-01,False
61,G47.3,Sleep apnea,AD,UKB,5-10,5-10 years before tenure,1.066715,0.652583,1.743658,7.967203e-01,16,1062,8.224210e-01,False
62,G47.3,Sleep apnea,MS,UKB,0,all,0.960653,0.454685,2.029654,9.162310e-01,7,3877,9.307743e-01,False


In [11]:
final_results.to_csv(f'SLEEP_Supplementary_Table_3_45_no_wales.csv', header = True, index = False)

In [12]:
!dx upload SLEEP_Supplementary_Table_3_45_no_wales.csv --path /results/SLEEP_Supplementary_Table_3_45_no_wales.csv

[===========================================================>] Uploaded 10,704 of 10,704 bytes (100%) SLEEP_Supplementary_Table_3_45_no_wales.csv
ID                          file-GjY83B0Jq9vy6jbjgJ6JKxq0
Class                       file
Project                     project-GZBqBx8Jq9vpQ6729F24BjYX
Folder                      /results
Name                        SLEEP_Supplementary_Table_3_45_no_wales.csv
State                       closing
Visibility                  visible
Types                       -
Properties                  -
Tags                        -
Outgoing links              -
Created                     Mon Apr 22 15:26:32 2024
Created by                  klevine22
 via the job                job-GjY73QjJq9vpgJ834g9k1bkb
Last modified               Mon Apr 22 15:26:32 2024
Media type                  
archivalState               "live"
cloudAccount                "cloudaccount-dnanexus"
